# Intento 11

- Uso de `RandomForestClassifier`.

In [1]:
import pandas as pd
import numpy as np
import time
from itertools import product
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# código común
import common.preprocessing as pre
from common.vars import SEED

# display all columns
pd.options.display.max_columns = None

In [2]:
train_features = pd.read_csv('../data/flu_training_set_features.csv', index_col='respondent_id')
train_labels = pd.read_csv('../data/flu_training_set_labels.csv', index_col='respondent_id')
test_features = pd.read_csv('../data/flu_test_set_features.csv', index_col='respondent_id')

In [3]:
def preprocess(train_features, test_features):
  """PREPROCESAMIENTO
  """
  labels = {}
  imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
  _train_features = train_features.astype(str)
  # uso SimpleImputer para "eliminar los nulos", como elimina los nombres de las columnas los ponemos de nuevo
  _train_features = pd.DataFrame(imp.fit_transform(_train_features), index=train_features.index)
  _train_features.columns = train_features.columns

  _all = pd.concat([train_features, test_features])

  # aprendo las etiquetas
  for col in _all.columns:
    labels[col] = LabelEncoder()
    labels[col].fit(_all[col].astype(str))
  
  # aplico el etiquetado
  for col in _train_features.columns:
    _train_features[col] = labels[col].transform(_train_features[col])
  
  _test_features = test_features.astype(str)
  _test_features = pd.DataFrame(imp.transform(_test_features), index=test_features.index)
  _test_features.columns = test_features.columns

  # aplico el mismo etiquetado con los valores de test
  for col in _test_features.columns:
    _test_features[col] = labels[col].transform(_test_features[col])
  
  return _train_features, _test_features

In [4]:
train_features, test_features = preprocess(train_features, test_features)

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

def cross_validation(model, X, y, cv):
  measures = []
  time_total_0 = time.time()

  for train, test in cv.split(X, y):
    X_train = X.iloc[train, :].values
    X_test = X.iloc[test, :].values
    y_train = y.iloc[train, :].values
    y_test = y.iloc[test, :].values
    time_iter_0 = time.time()
    _model = model.fit(X_train, y_train)
    time_iter = time.time() - time_iter_0
    preds = _model.predict_proba(X_test)
    y_preds = pd.DataFrame(
      {
        'h1n1_vaccine':     preds[0][:,1],
        'seasonal_vaccine': preds[1][:,1],
      },
      index = y.iloc[test,:].index
    )
    measure = roc_auc_score(y_test, y_preds)
    print(f"[CV] AUC score: {measure:.4f}. Time: {time_iter:6.2f}")
    measures.append(measure)
  
  time_total = time.time() - time_total_0
  print(f"[CV] Average AUC: {np.mean(measures):.4f}. Total time: {time_total:6.2f}")
  return model, y_preds

def preds_to_df(preds, features):
  return pd.DataFrame(
    {
      'h1n1_vaccine': preds[0][:,1],
      'seasonal_vaccine': preds[1][:,1]
    },
    index = features.index
  )

### _Holdout_ como proxy de ROC en test

In [6]:
def holdout_proxy(model, X, y, test_size=0.2):
  # split train features in two
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)

  model_cvd, _ = cross_validation(model, X_train, y_train, kf)

  # aprendemos con todos los ejemplos
  model_cvd = model_cvd.fit(X_train, y_train)
  # aplicamos probabilidad
  preds = model_cvd.predict_proba(X_test)

  return roc_auc_score(y_test, preds_to_df(preds, X_test))

In [7]:
import random
def proxy_search(search_space, model_lambda, X, y, test_size=0.2, limit=20):
  space = list(dict(zip(search_space.keys(), values)) for values in product(*search_space.values()))
  n = len(list(product(*search_space.values())))
  results = []
  i = 1
  while i <= limit:
    params = space[random.randint(0, n-1)]
    print(f"[Proxy Search] ({i} of {limit}) Searching with params {params}")
    model = model_lambda(params)
    roc_auc = holdout_proxy(model, X, y, test_size=test_size)
    results.append({
      'params': params,
      'roc_auc': roc_auc
    })

    i += 1
  
  return results

In [8]:
rf_multi_lambda = lambda params: MultiOutputClassifier(RandomForestClassifier(random_state=SEED, **params))

In [9]:
search_space = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [10]:
search = proxy_search(search_space, rf_multi_lambda, train_features, train_labels)

[Proxy Search] (1 of 20) Searching with params {'bootstrap': True, 'max_depth': 60, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1000}
[CV] AUC score: 0.8626. Time:  34.28
[CV] AUC score: 0.8653. Time:  67.27
[CV] AUC score: 0.8548. Time:  72.24
[CV] AUC score: 0.8518. Time:  67.19
[CV] AUC score: 0.8669. Time:  69.61
[CV] Average AUC: 0.8603. Total time: 331.66
[Proxy Search] (2 of 20) Searching with params {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 1400}
[CV] AUC score: 0.8602. Time:  35.61
[CV] AUC score: 0.8636. Time:  41.18
[CV] AUC score: 0.8535. Time:  40.91
[CV] AUC score: 0.8503. Time:  35.65
[CV] AUC score: 0.8645. Time:  33.84
[CV] Average AUC: 0.8584. Total time: 199.25
[Proxy Search] (3 of 20) Searching with params {'bootstrap': False, 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 600}
[CV] AU

In [11]:
search_max = max(search, key=lambda item: item['roc_auc'])
search_max

{'params': {'bootstrap': True,
  'max_depth': 50,
  'max_features': 'auto',
  'min_samples_leaf': 4,
  'min_samples_split': 5,
  'n_estimators': 1800},
 'roc_auc': 0.8651691969281137}